In [1]:
pip install rtdl_revisiting_models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [2]:
pip install pytorch_tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ===============================================================
# HIGGS BOSON — TabNetEncoder + SupCon (Joint Training, t-SNE After)
# ===============================================================

import os, math, zipfile
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ---- fixed: use TabNetEncoder (safe across pytorch_tabnet versions)
from pytorch_tabnet.tab_network import TabNetEncoder
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# ---------------- Settings ----------------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

BATCH_SIZE = 512
EPOCHS = 40
LR = 7e-4
WEIGHT_DECAY = 3e-5
PROJ_DIM = 64

SUPCON_TEMPERATURE = 0.08
HARD_NEG_TOPK = 30
INITIAL_SUP_W, MAX_SUP_W = 0.12, 0.45

# TabNet hyperparams
TABNET_N_D = 64
TABNET_N_A = 64
TABNET_N_STEPS = 5
TABNET_GAMMA = 1.5
TABNET_LAMBDA_SPARSE = 1e-3
N_FOLDS = 5

# ---------------- Kaggle File Handling ----------------
zip_files = {
    "train": "/kaggle/input/higgs-boson/training.zip",
    "test":  "/kaggle/input/higgs-boson/test.zip",
    "submission": "/kaggle/input/higgs-boson/random_submission.zip"
}
extract_dir = "/kaggle/working/higgs_data/"
os.makedirs(extract_dir, exist_ok=True)
for key, path in zip_files.items():
    if os.path.exists(path):
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(extract_dir)
            print(f"{key} unzipped.")
    else:
        print(f"{key} zip not found at {path}")

TRAIN_CSV = os.path.join(extract_dir, "training.csv")
TEST_CSV  = os.path.join(extract_dir, "test.csv")
OUT_SUB  = "/kaggle/working/submission.csv"

# ---------------- AMS Metric ----------------
def ams_score(s, b):
    b_reg = 10.0
    rad = 2.0 * ((s + b + b_reg) * math.log(1.0 + s / (b + b_reg)) - s)
    return math.sqrt(rad) if rad > 0 else 0.0

# ---------------- Dataset ----------------
class HiggsDataset(Dataset):
    def __init__(self, X, y=None, sample_weight=None):
        self.X = X.astype(np.float32)
        self.y = y
        self.sample_weight = sample_weight
    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        if self.y is None:
            return x, torch.tensor(-1, dtype=torch.long), torch.tensor(0.0)
        y = torch.tensor(int(self.y[idx]), dtype=torch.long)
        w = torch.tensor(float(self.sample_weight[idx]) if self.sample_weight is not None else 1.0)
        return x, y, w

# ---------------- SupCon Loss (Hard-negative mining) ----------------
class HardNegSupConLoss(nn.Module):
    def __init__(self, temperature=0.08, top_k=30):
        super().__init__()
        self.temperature = temperature
        self.top_k = top_k
    def forward(self, features, labels):
        device = features.device
        labels = labels.contiguous().view(-1,1)
        bs = features.shape[0]
        sim = torch.div(torch.matmul(features, features.T), self.temperature)
        same = torch.eq(labels, labels.T).float().to(device)
        diag = torch.eye(bs, device=device)
        pos_mask = same - diag
        neg_mask = 1.0 - same
        sim_max, _ = torch.max(sim, dim=1, keepdim=True)
        sim_stable = sim - sim_max.detach()
        exp_sim = torch.exp(sim_stable)
        pos_exp = exp_sim * pos_mask
        neg_sim_masked = sim.clone()
        neg_sim_masked[neg_mask == 0] = -1e9
        eps = 1e-12
        k = min(self.top_k, max(1, int(neg_mask.sum(dim=1).max().item())))
        if k == 0:
            denom = pos_exp.sum(dim=1) + (exp_sim*(1.0 - diag - pos_mask)).sum(dim=1)
            log_prob = torch.log((pos_exp.sum(dim=1)+eps)/(denom+eps))
            valid = pos_mask.sum(dim=1)>0
            return -log_prob[valid].mean()
        topk_vals, topk_idx = torch.topk(neg_sim_masked, k=k, dim=1)
        sel_mask = torch.zeros_like(neg_mask)
        arange = torch.arange(bs, device=device).unsqueeze(1).expand(-1,k)
        sel_mask[arange.reshape(-1), topk_idx.reshape(-1)] = 1.0
        sel_mask = sel_mask * neg_mask
        sum_pos = pos_exp.sum(dim=1)
        sum_neg = (exp_sim * sel_mask).sum(dim=1)
        log_prob = torch.log((sum_pos + eps)/(sum_pos + sum_neg + eps))
        valid = pos_mask.sum(dim=1)>0
        return -log_prob[valid].mean()

class TabNetSupCon(nn.Module):
    def __init__(self, n_features, proj_dim=64):
        super().__init__()
        try:
            self.encoder = TabNetEncoder(
                input_dim=n_features,
                output_dim=TABNET_N_D,
                n_d=TABNET_N_D,
                n_a=TABNET_N_A,
                n_steps=TABNET_N_STEPS,
                gamma=TABNET_GAMMA,
                cat_idxs=[],
                cat_dims=[],
                cat_emb_dim=[],
                n_independent=2,
                n_shared=2,
                epsilon=1e-15,
                virtual_batch_size=128,
                momentum=0.02
            )
        except TypeError:
            self.encoder = TabNetEncoder(
                input_dim=n_features,
                output_dim=TABNET_N_D,
                n_d=TABNET_N_D,
                n_a=TABNET_N_A,
                n_steps=TABNET_N_STEPS,
                gamma=TABNET_GAMMA,
                epsilon=1e-15,
                virtual_batch_size=128,
                momentum=0.02
            )

        self.classifier = nn.Linear(TABNET_N_D, 2)
        self.proj_head = nn.Sequential(
            nn.Linear(TABNET_N_D, 128), nn.ReLU(), nn.Linear(128, proj_dim)
        )

    def forward(self, x, return_emb=False):
        device = x.device

    # ensure everything is on the same device
        self.encoder.to(device)
        self.classifier.to(device)
        self.proj_head.to(device)

        if hasattr(self.encoder, "group_attention_matrix"):
            self.encoder.group_attention_matrix = self.encoder.group_attention_matrix.to(device)

        if hasattr(self.encoder, "initial_bn"):
            bn = self.encoder.initial_bn
            if isinstance(bn, nn.BatchNorm1d):
                bn.to(device)
            elif isinstance(bn, (list, nn.ModuleList, tuple)):
                for block in bn:
                    block.to(device)

    # forward through TabNet encoder
        out_tuple = self.encoder(x)

    # handle output structure
        if isinstance(out_tuple, (list, tuple)):
            out_raw = out_tuple[0]
        # case 1: out_raw is a list of step tensors -> average them
            if isinstance(out_raw, list):
                out = torch.stack(out_raw, dim=0).mean(0)
        # case 2: out_raw is a tensor already
            elif torch.is_tensor(out_raw):
                out = out_raw
            else:
                raise TypeError(f"Unexpected inner type from TabNetEncoder: {type(out_raw)}")
        elif torch.is_tensor(out_tuple):
            out = out_tuple
        else:
            raise TypeError(f"Unexpected output type from TabNetEncoder: {type(out_tuple)}")

    # classifier and projection head
        logits = self.classifier(out)
        emb = F.normalize(self.proj_head(out), dim=1)

        return (logits, emb) if return_emb else logits

# ---------------- Weighted Focal Loss ----------------
class WeightedFocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, logits, targets, sample_weight=None):
        ce = F.cross_entropy(logits, targets, reduction='none')
        p_t = torch.exp(-ce)
        focal = ((1 - p_t) ** self.gamma) * ce
        if sample_weight is not None:
            focal = focal * sample_weight
            return focal.sum() / (sample_weight.sum() + 1e-12)
        return focal.mean()

# ---------------- Load Data ----------------
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

train_df.replace(-999.0, np.nan, inplace=True)
test_df.replace(-999.0, np.nan, inplace=True)

for c in train_df.columns:
    if c in ['EventId','Weight','Label']: continue
    if train_df[c].isna().any():
        train_df[c+'_miss'] = train_df[c].isna().astype(int)
        test_df[c+'_miss']  = test_df[c].isna().astype(int)

numeric_cols = [c for c in train_df.select_dtypes(include=np.number).columns if c != "Weight"]
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
num_cols_test = [c for c in numeric_cols if c in test_df.columns]
test_df[num_cols_test] = test_df[num_cols_test].fillna(train_df[num_cols_test].median())

if {'DER_mass_MMC','DER_mass_vis'}.issubset(train_df.columns):
    train_df['mass_ratio'] = train_df['DER_mass_MMC']/(train_df['DER_mass_vis']+1e-6)
    test_df['mass_ratio']  = test_df['DER_mass_MMC']/(test_df['DER_mass_vis']+1e-6)
if {'PRI_tau_pt','PRI_met'}.issubset(train_df.columns):
    train_df['pt_ratio'] = train_df['PRI_tau_pt']/(train_df['PRI_met']+1e-6)
    test_df['pt_ratio']  = test_df['PRI_tau_pt']/(test_df['PRI_met']+1e-6)

y = (train_df['Label'] == 's').astype(int).values
weights = train_df['Weight'].values
event_ids_test = test_df['EventId'].values

train_features = train_df.drop(columns=['EventId','Weight','Label'], errors='ignore')
test_features  = test_df.drop(columns=['EventId'], errors='ignore')

scaler = StandardScaler()
X = scaler.fit_transform(train_features.values.astype(np.float32))
X_test = scaler.transform(test_features.values.astype(np.float32))

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# ---------------- Training ----------------
oof_nn = np.zeros(len(X))
test_pred_nn = np.zeros(len(X_test))

supcon = HardNegSupConLoss(temperature=SUPCON_TEMPERATURE, top_k=HARD_NEG_TOPK)
tsne_embs_before, tsne_embs_after, tsne_labels = [], [], []

for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
    print(f"\n===== Fold {fold+1}/{N_FOLDS} =====")
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    w_tr, w_va = weights[tr_idx], weights[va_idx]
    tr_loader = DataLoader(HiggsDataset(X_tr, y_tr, w_tr), batch_size=BATCH_SIZE, shuffle=True)
    va_loader = DataLoader(HiggsDataset(X_va, y_va, w_va), batch_size=BATCH_SIZE, shuffle=False)
    model = TabNetSupCon(X.shape[1], PROJ_DIM).to(DEVICE)

    # t-SNE "before"
    model.eval()
    emb_before = []
    with torch.no_grad():
        for xb,_,_ in va_loader:
            xb = xb.to(DEVICE)
            _, emb = model(xb, return_emb=True)
            emb_before.append(emb.cpu().numpy())
    tsne_embs_before.append(np.vstack(emb_before))
    tsne_labels.append(y_va)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    criterion_cls = WeightedFocalLoss(alpha=0.75, gamma=2.0)
    best_auc, best_state, best_probs = 0, None, None

    for epoch in range(EPOCHS):
        model.train()
        sup_w = INITIAL_SUP_W + (MAX_SUP_W - INITIAL_SUP_W) * (epoch / max(1, EPOCHS-1))
        for xb, yb, wb in tqdm(tr_loader, desc=f"Fold{fold+1} Epoch{epoch+1}", leave=False):
            xb, yb, wb = xb.to(DEVICE), yb.to(DEVICE), wb.to(DEVICE)
            optimizer.zero_grad()
            logits, emb = model(xb, return_emb=True)
            loss_cls = criterion_cls(logits, yb, wb)
            loss_sup = supcon(emb, yb)
            loss = loss_cls + sup_w * loss_sup
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        val_probs = []
        with torch.no_grad():
            for xb,_,_ in va_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                p = F.softmax(logits, dim=1)[:,1].cpu().numpy()
                val_probs.extend(p)
        val_auc = roc_auc_score(y_va, val_probs)
        print(f"Fold {fold+1} Epoch {epoch+1} AUC: {val_auc:.5f}")
        if val_auc > best_auc:
            best_auc = val_auc
            best_state = {k:v.cpu() for k,v in model.state_dict().items()}
            best_probs = val_probs.copy()

    if best_state is not None:
        model.load_state_dict(best_state)
    oof_nn[va_idx] = np.array(best_probs)

    # t-SNE "after"
    model.eval()
    emb_after = []
    with torch.no_grad():
        for xb,_,_ in va_loader:
            xb = xb.to(DEVICE)
            _, emb = model(xb, return_emb=True)
            emb_after.append(emb.cpu().numpy())
    tsne_embs_after.append(np.vstack(emb_after))

    # Test predictions
    test_loader = DataLoader(HiggsDataset(X_test), batch_size=1024, shuffle=False)
    preds = []
    with torch.no_grad():
        for xb,_,_ in test_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            p = F.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds.extend(p)
    test_pred_nn += np.array(preds) / N_FOLDS

# ---------------- t-SNE Visualization ----------------
X_after  = np.vstack(tsne_embs_after)
y_tsne   = np.concatenate(tsne_labels)
print("Running t-SNE on final embeddings (after training)...")
tsne = TSNE(n_components=2, perplexity=50, random_state=SEED, init='pca', n_iter=1000)
X2d_after = tsne.fit_transform(X_after)
plt.figure(figsize=(7,6))
plt.scatter(X2d_after[:,0], X2d_after[:,1], c=y_tsne, s=6, alpha=0.7)
plt.title("t-SNE of Validation Embeddings — AFTER (SupCon + TabNetEncoder)")
plt.xlabel("TSNE-1"); plt.ylabel("TSNE-2")
plt.show()

# ---------------- AMS Optimization ----------------
thr_range = np.linspace(0.01,0.99,99)
best_thr, best_ams = 0.5, -1
for t in thr_range:
    s = weights[(y==1) & (oof_nn>t)].sum()
    b = weights[(y==0) & (oof_nn>t)].sum()
    sc = ams_score(s,b)
    if sc > best_ams:
        best_ams, best_thr = sc, t
print(f"Best AMS on OOF = {best_ams:.3f} @ thr={best_thr:.4f}")

# ---------------- Submission ----------------
print("\nWriting submission...")
rankorder = np.argsort(np.argsort(test_pred_nn)) + 1
classes = np.where(test_pred_nn > best_thr, 's', 'b')
sub = pd.DataFrame({"EventId": event_ids_test, "RankOrder": rankorder, "Class": classes})
sub.to_csv(OUT_SUB, index=False)
print("Saved submission to:", OUT_SUB)
print("Final AMS:", best_ams)


train unzipped.
test unzipped.
submission unzipped.

===== Fold 1/5 =====


Fold 1 Epoch 1 AUC: 0.59743


Fold 1 Epoch 2 AUC: 0.62275


Fold 1 Epoch 3 AUC: 0.66753


Fold 1 Epoch 4 AUC: 0.69742


...

Fold 1 Epoch 10 AUC: 0.81427


Fold 1 Epoch 11 AUC: 0.81945


Fold 1 Epoch 12 AUC: 0.82621


Fold 1 Epoch 13 AUC: 0.82985


Fold1 Epoch14:   8%|▊         | 30/391 [00:01<00:19, 18.67it/s]